In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Procesamiento de Pliegos de Licitación

Este notebook demuestra el flujo completo para:
1. Extraer texto de PDFs de pliegos de licitación
2. Procesar y analizar el contenido
3. Identificar secciones clave y entidades importantes
4. Generar un reporte estructurado

### Configuración Inicial

In [9]:
import fitz  # PyMuPDF
import pdfplumber
import pytesseract # OCR
from PIL import Image
import io
import re
import os
import sys
import importlib
import traceback
import os
from pathlib import Path
import json
import datetime
from typing import Dict, List

In [10]:
from src.utils.pdf_extractor import PDFTextExtractor, extract_text_from_pdf
import src.utils.pdf_extractor as pdf_extractor_mod
importlib.reload(pdf_extractor_mod) 

<module 'src.utils.pdf_extractor' from 'c:\\users\\nori\\programmingprojects\\hackathons\\hackiathon_neurobit_licitacion\\src\\utils\\pdf_extractor.py'>

In [11]:
# Agregar src al path para importar nuestros módulos
module_path = str(Path.cwd().parent / "src")
if module_path not in sys.path:
    sys.path.append(module_path)

#### Ejemplo con OCR a un archivo txt

Verificar que `tesseract.exe` y `spa.traineddata` esten instalados.

In [12]:
tess_base = r"C:\Program Files\Tesseract-OCR"   # ajusta si lo instalaste en otra carpeta
print("tesseract.exe exists:", os.path.exists(os.path.join(tess_base, "tesseract.exe")))
print("spa.traineddata exists:", os.path.exists(os.path.join(tess_base, "tessdata", "spa.traineddata")))

tesseract.exe exists: True
spa.traineddata exists: True


In [13]:

pdf_path = "../data/PLIEGO-LICO-V-2023-001.pdf"
out_folder = "../2.1_results"
os.makedirs(out_folder, exist_ok=True)

out_file = os.path.join(out_folder, f"result.txt")

try:
    extractor = PDFTextExtractor()
    result = extractor.extract_text(pdf_path, use_ocr=True, extract_tables=True)

    with open(out_file, "w", encoding="utf-8") as f:
        f.write("=== TEXTO EXTRAÍDO ===\n\n")
        f.write(result.get('text', '') or "")
        f.write("\n\n=== METADATOS ===\n")
        metadata = result.get('metadata', {})
        for k, v in metadata.items():
            f.write(f"{k}: {v}\n")

        tables = result.get('tables', [])
        if tables:
            f.write("\n\n=== TABLAS DETECTADAS ===\n")
            for i, table in enumerate(tables, start=1):
                f.write(f"\nTabla {i} (Página {table.get('page', '?')}):\n")
                for row in table.get('table', []):
                    # aseguramos cada celda como string y evitamos None
                    safe_row = [("" if c is None else str(c)) for c in row]
                    f.write(" | ".join(safe_row) + "\n")

    print(f"Extracción completada. Resultado guardado en: {out_file}")

except Exception as e:
    print("ERROR durante la extracción:")
    print(str(e))
    print("\n⤷ TRACEBACK COMPLETO:")
    traceback.print_exc()
    print("\nConsejo rápido: si ves PermissionError, cierra cualquier visor del PDF (Acrobat/Edge).")
    print("Si ves errores de Tesseract sobre 'spa.traineddata', revisa que esté instalado o usa fallback en el módulo.")


Extracción completada. Resultado guardado en: ../2.1_results\result.txt
